In [ ]:
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_dir = 'Dataset/'
target_size = (244, 244)
classes = 3

train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.1,
                                   horizontal_flip=False,
                                   validation_split=0.2)
                                   
train_gen = train_datagen.flow_from_directory(data_dir, target_size = target_size,
                                             shuffle=True,
                                             batch_size=32,
                                             color_mode = 'rgb',
                                             class_mode = 'categorical',
                                             subset='training',
                                             )


val_gen = train_datagen.flow_from_directory(data_dir, target_size=target_size, batch_size=32,
                                           color_mode= 'rgb',
                                           class_mode = 'categorical',
                                           subset= 'validation')


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=3, strides=1, activation='relu', padding='same', input_shape=(244, 244, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

model.add(Conv2D(64, kernel_size=3, strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(64, kernel_size=3, strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(128, kernel_size=3, strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=1))

model.add(Flatten())

model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(Dense(classes, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_gen, epochs=5, validation_data=val_gen)

In [ ]:
model_json = model.to_json()
with open("model.json", 'w') as file:
    file.write(model_json)
model.save_weights("model.weights.h5")
print("SUCCESSFUL model is SAVED")

In [ ]:

from keras.models import model_from_json
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()

load_model = model_from_json(loaded_model_json)
load_model.load_weights("model.weights.h5")
print("LOADED")

In [ ]:
import numpy as np
import cv2

# Memuat gambar
image = cv2.imread("Examples/ex6.jpg")

# Mengubah ukuran gambar menjadi 100x100 piksel
image = cv2.resize(image, (244, 244))

# Mengubah bentuk gambar menjadi (1, 100, 100, 3) agar sesuai dengan input model
image = np.reshape(image, (1, 244, 244, 3))

# Melakukan prediksi dengan model yang sudah dimuat
prediksi = load_model.predict(image)

# Menampilkan hasil prediksi

print("Prediksinya adalah angka", prediksi)

print("Prediksinya adalah angka", np.argmax(prediksi))

if(np.argmax(prediksi) == 0) :
    print("Prediksinya adalah hawar")
if(np.argmax(prediksi) == 1) :
    print("Prediksinya adalah sehat")
if(np.argmax(prediksi) == 2) :
    print("Prediksinya adalah karat")